In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import time
from torch.optim.lr_scheduler import StepLR
import sympy as sp

In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [4]:
import pandas as pd
df = pd.read_csv('FeynmanEquations.csv')

functions = []
num_vars_per_func = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# variables']
    function_details = {
        'formula': formula,
        'variables': []
    }
    
    for i in range(1, 11):  
        v_name = row.get(f'v{i}_name', None)
        v_low = row.get(f'v{i}_low', None)
        v_high = row.get(f'v{i}_high', None)
        
        if pd.notna(v_name):
            function_details['variables'].append({
                'name': v_name,
                'low': v_low,
                'high': v_high
            })
    
    functions.append(function_details)
    num_vars_per_func.append(num_vars)

'''for i, func in enumerate(functions):
    print(f"Function {i+1}:")
    print(f"  Formula: {func['formula']}")
    print(f"  Number of Variables: {num_vars_per_func[i]}")
    print("  Variables:")
    for var in func['variables']:
        print(f"    - Name: {var['name']}, Range: ({var['low']}, {var['high']})")
    print()'''

'for i, func in enumerate(functions):\n    print(f"Function {i+1}:")\n    print(f"  Formula: {func[\'formula\']}")\n    print(f"  Number of Variables: {num_vars_per_func[i]}")\n    print("  Variables:")\n    for var in func[\'variables\']:\n        print(f"    - Name: {var[\'name\']}, Range: ({var[\'low\']}, {var[\'high\']})")\n    print()'

In [3]:
def generate_function(function, sample_size, device):
    hold = []
    sympy_symbols = []
    
    for var in function["variables"]:
        sym = sp.symbols(var["name"])
        sympy_symbols.append(sym)
        min_val = var["low"]
        max_val = var["high"]
        v = (max_val - min_val) * torch.rand(sample_size, 1, 1) + min_val
        hold.append(v)
    
    params = torch.stack(hold)
    params = torch.atleast_2d(params)
    params = torch.transpose(params, 0, 1)
    params = torch.transpose(params, 1, 2)
    params = params.to(device)

    formula = sp.sympify(function["formula"])
    
    results = []
    for i in range(sample_size):
        var_values = {sympy_symbols[j]: params[i, 0, j].item() for j in range(len(sympy_symbols))}
        evaluated = formula.subs(var_values)
        results.append(float(evaluated))
    
    return results, formula, sympy_symbols, params

In [5]:
sample_size = 1000
hold = []
for f in functions[0:10]:
    try:
        results = generate_function(f, sample_size, device)
        hold.append(results)
    except Exception as e:
        print(f"Error processing function {f}: {e}")

In [8]:
hold_data = [torch.tensor(l[0]) for l in hold]
data = torch.stack(hold_data).reshape(-1)
hold_formulas = [l[1] for l in hold]
hold_symbols = [l[2] for l in hold]
hold_params = [l[3] for l in hold]
hold_num_params = [len(l[2]) for l in hold]

In [9]:
def compute_derivative(parameters, func, symbols, formula, epsilon=1e-6):
    batch_size = parameters.shape[0]
    num_params = parameters.shape[2]
    gradients = torch.zeros(batch_size, num_params)

    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(num_params):
            perturbed_params_pos = param_tensor.clone()
            perturbed_params_neg = param_tensor.clone()
            
            perturbed_params_pos[0, j, 0] += epsilon
            forward_value = func(perturbed_params_pos, symbols, formula)
            
            perturbed_params_neg[0, j, 0] -= epsilon
            backward_value = func(perturbed_params_neg, symbols, formula)
            
            gradients[i, j] = (forward_value - backward_value) / (2 * epsilon)
    
    return gradients


In [10]:
def compute_hessians(parameters, func, symbols, formula, epsilon=1e-6): #need to fix
    batch_size = parameters.shape[0]
    max_num_params = parameters.shape[2]
    hessians = torch.zeros((batch_size, max_num_params, max_num_params))
    
    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(max_num_params):
            for k in range(max_num_params):
                # Perturb j-th and k-th parameters
                perturbed_params = param_tensor.clone()
                
                # Compute f(x + epsilon * e_j + epsilon * e_k)
                perturbed_params_jk = perturbed_params.clone()
                perturbed_params_jk[:, j, :] += epsilon
                perturbed_params_jk[:, k, :] += epsilon
                f_plus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x + epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_plus_minus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j + epsilon * e_k)
                perturbed_params_jk[:, j, :] -= 2 * epsilon
                perturbed_params_jk[:, k, :] += 2 * epsilon
                f_minus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_minus_minus = func(perturbed_params_jk, symbols, formula)
                hessians[i, j, k] = (f_plus_plus - f_plus_minus - f_minus_plus + f_minus_minus) / (4 * epsilon**2)
    
    return hessians

In [11]:
def evaluate_function(params, symbols, formula):
    if params.shape[0] != 1:
        var_values = {symbols[j]: params[j].item() for j in range(len(symbols))}
    else:
        var_values = {symbols[j]: params[:, j, :].item() for j in range(len(symbols))}
    evaluated = formula.subs(var_values)
    return float(evaluated)

In [12]:
derivatives = [compute_derivative(hold_params[i], evaluate_function, hold_symbols[i], hold_formulas[i]) for i in range(len(hold_params))]
hessians = [compute_hessians(hold_params[i], evaluate_function, hold_symbols[i], hold_formulas[i]) for i in range(len(hold_params))]

In [17]:
torch.save({
    'data': data,
    'formulas': hold_formulas,
    'symbols': hold_symbols,
    'params': hold_params,
    'num_params': hold_num_params,
    'derivatives': derivatives,
    'hessians': hessians
}, 'hold_data.pth')